In [1]:
import pandas as pd
import numpy as np

print('a')


a


In [2]:

#makes it so that when you print the data frame, the entire data frame is displayed
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

In [4]:
# loading database of airport codes with location
# source: datahub, airport codes, https://datahub.io/core/airport-codes#python
airports_df = pd.read_csv('airportCodesTypeFiltered.csv')

# loading origin and destination of flights taken by bates employees
travel_df_init = pd.read_excel("JPM Airline Activity 7.1.22 - 6.30.23 Data Conversion.xlsx")
 #creating two empty dataframes for origin and destination
#IATA codes
#travel_df_init
airports_df.head()

,Unnamed: 0,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,0,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
1,1,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
2,2,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
3,4,00AS,small_airport,Fulton Airport,1100.0,NaN,US,US-OK,Alex,00AS,NaN,00AS,"-97.8180194, 34.9428028"
4,5,00AZ,small_airport,Cordes Airport,3810.0,NaN,US,US-AZ,Cordes,00AZ,NaN,00AZ,"-112.16500091552734, 34.305599212646484"


In [15]:
def cleaning(travel_df_init):
    '''
        The purpose of this function is to perform three cleaning operations given a travel data file
        inputed as a data frame.

        Inputs:
            travel_df: a data frame containing the travel file information in its entirity
            origins_df: a data frame with just the origination column of the travel file
            dest_df: a data frame containing just the destination column of the travel file

        Returns:
            an updated data frame with redundant and false data redacted
    '''
    travel_copy = travel_df_init.copy()
    travel_df_init["Comparison"] = travel_df_init["Origination"] == travel_df_init["Destination"]
    #creating a new column in the travel data frame  with just the origin with true/False
    #depending on where the origination entry in each row is equal to the destination entry in
    #the same row
    #print(travel_df_init.where(travel_df_init["Comparison"] == True))

    if travel_df_init["Comparison"].any() == True: #if any value in the comparison column is true
        ##BROKENNN!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        orig_eq_dest_row = (travel_copy[travel_copy["Origination"] == travel_copy["Destination"]].index.values)
        #orig_eq_dest_row represents the row in the travel file copy where the Origin and Destination
        #are the same for a row
        #travel_copy = (travel_copy.drop(orig_eq_dest_row, axis = 0))
        travel_copy = travel_copy.drop(orig_eq_dest_row)
        print(travel_copy["Referenced Transaction Amount"][0])
        travel_copy = travel_copy[travel_copy['Referenced Transaction Amount'] >= 90]

    #if travel_df_init["Ticket Number"].isna().any() == True:
    #figure out how to trop rows where this is the case
    #travel_copy = travel_copy.drop_duplicates(subset = ["Origination", "Destination", "Ticket Number"])
    #duplicate_df = travel_copy.drop_duplicated(subset = ["Origination", "Destination", "Ticket Number"])
    #print(duplicate_df.shape[0])
    #print(result_df.join(travel_copy))

    #KYLE: THIS MIGHT BE UNNECESSARY 10/10/2023 travel_copy = travel_copy.drop_duplicates(subset = ['Origination', 'Destination', 'Ticket Number'])

    #print(travel_copy.drop_duplicates(subset = ['Origination', 'Destination', 'Ticket Number']))


    #drop the second row (in terms of appearance) where the origination, destination, and ticket number
    #are the smame
    #display(travel_copy)
    return travel_copy

flights_df = cleaning(travel_df_init)
#display(flights_df.shape[0])
#display(flights_df)
#print(flights_df)
#print(f"The new travel data frame is")
#display(flights_df)
origins_df, destination_df = pd.DataFrame(), pd.DataFrame()
origins_df['Origination'] = flights_df['Origination'] #Taking the origin column from the travel file and putting it into
#the new empty data frame
destination_df['Destination'] = flights_df['Destination']

#to test, we are going to compare the length of this data frame to the number of columns in the original file - the number of problematic rows in the testing file.

321.98


In [16]:

def find_airport(airport_code):
    '''
    input:
    airport_code: IATO code for an airport
    returns:
    df: dataframe or series filtered by the given airport_code
    '''

    '''
    df = airports_df[airports_df['ident'].str.endswith(f'{airport_code}')] #this gives us the row in airport codes where the airport
    #in the origin/destination dataframe's airport is found. It looks at the ident row which corresponds to IACA codes then
    #returns the whole row where the airport is found at the end of the ident string
    '''
    #print(f'The airport code in the Origination/Destination column is {airport_code}')
    df = airports_df[airports_df['iata_code'].str.contains(airport_code, na = False)] #add lines to the new data frame which contain
    #the identified IATA code and replace unknown elements (hence, na = False) in the line with NaN
    return df



def find_coordinates(airport_code) -> list:
    '''
    input:
    airport_code: IATO code for an airport
    returns:
    coord: longitude and latitude corresponding to the given airport

    '''

    airport = find_airport(airport_code)
    if len(airport.index) == 0:
      return np.nan


    coord =  airport['coordinates'].values #gets the coordinate pair for each row that is identified to be associated with
    #a particular IATA code; note that this means we can have multiple entries associated with the coord variable for a single
   #airport such that coord > 1; coord is an array
    #print(f"coord is {coord}")

    if coord.size > 1:
        countries = airport['iso_country'].values
        if 'US' in countries:
            coord = airport[airport['iso_country'] == 'US']['coordinates'].values #select the coord with US for an iso_country
        else:
            display(airport)
            correct_name = input("Please enter the name of the correct airport")
            coord = airport[airport['name'] == correct_name]['coordinates'].values

    if coord.size == 0:
        return np.nan

    coord = [float(axis) for axis in coord[0].split(',')] #splits each string pair at the comma then converts the entries to a
    #float; returns a list of floats

    return coord


In [18]:
# finding coordinates of origin and destination airports
origins_df['coordinates'] = origins_df['Origination'].apply(find_coordinates) #this means that origins_df is treated
#as the airport code in def find_coordinates; since find_coordinates and find_airport are in the same block, find_airport also
#gets access to airport_code# I guess by default the apply function works down each row
destination_df['coordinates'] = destination_df['Destination'].apply(find_coordinates)

#display(origins_df)
#display(destination_df)
print(len(origins_df))
origins_df.dropna(axis = 0, how = "any", inplace = True ) #dropping rows for which airports could not be identified
destination_df.dropna(axis = 0, how = "any", inplace = True)
#display(origins_df)

# extracting latitude and longitude data from coordinates
origins_df['long'] = origins_df['coordinates'].apply(lambda c: c[0]) #essentially, take the origins_df coordinate column,
#which currently contains lattitude and longitude information, grab the longitude information, and assign it to a new
#column called 'long' which refers to origination airport longitude data
origins_df['lat'] = origins_df['coordinates'].apply(lambda c: c[1])
destination_df['long'] = destination_df['coordinates'].apply(lambda c: c[0])
destination_df['lat'] = destination_df['coordinates'].apply(lambda c: c[1])

2664


In [8]:
# checkinig that all observations have exactly one lat/long coordinate
origins_df[origins_df['coordinates'].apply(lambda c: True if len(c) == 2 else False)].size/4 == origins_df.size/4

True

In [9]:
def find_dist(locs) -> float:
    '''
    input:
    locs: dataframe consisting of origin and destination locations in lat/long coordinates
    returns:
    flight_distance: distance between two sets of lat/long coordinates as calculated usig haversine formula
    '''
    lat1, lon1, lat2, lon2 = np.radians(
        [locs['o_lat'], locs['o_long'], locs['d_lat'], locs['d_long']])

    earth_radius = 6371 # 6.3781×106 m
    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2
    flight_distance = earth_radius * 2 * np.arcsin(np.sqrt(a))

    return flight_distance

In [10]:
# creating new datafram for calculating distance
dist_df = pd.DataFrame()
dist_df[['Origination','o_long','o_lat']] = origins_df[['Origination','long','lat']]
dist_df[['Destination','d_long','d_lat']] = destination_df[['Destination','long','lat']]

# finding distance
dist_df['distance (km)'] = dist_df.apply(find_dist, axis=1) #axis = 1 means move across the colums to grab data for find_dist
dist_df['distance (miles)'] = 0.621371 * dist_df['distance (km)']

# sum of all distances in miles
total_dist = dist_df['distance (miles)'].sum()
print(f'Total flight distance travelled is {total_dist:.1f} miles.') #.1f reduces precision to 1 digit; hence result
#prints with 1 decimal point

Total flight distance travelled is 2714250.9 miles.


In [11]:
def find_type(distance):
    '''
    input:
    distance: distance between origin and destination in miles
    return:
    flight type based on lenght: short, med, or long
    '''
    short, long = 300, 2300
    if distance <= short:
        return 'short'
    elif distance <= long:
        return 'med'
    else:
        return 'long'


In [12]:
# catagorizing flight as short, med or long
dist_df['type'] = dist_df['distance (miles)'].apply(find_type)

In [13]:
def emission_factor(flight_type):
    '''
    input:
    flight_type: short, med, long
    return:
    factor: CO2 per passenger per mile
    '''
    # source: Emission Factors for Greenhouse Gas Inventories, US. Environmental protection Agency in ton CO2/pass mile (Biruk)
    factor = {'short': 0.0002091268, 'med': 0.0001302368, 'long': 0.0001645646} #UPDATED AS OF 10/17/2023
    return factor[flight_type] #essentially, after the factor is found, return the correct factor given the input flight type


In [14]:
# assigning emision factor based on flight distance type
dist_df['emission_factor'] = dist_df['type'].apply(emission_factor)

# calculating emission for each flight
dist_df['emission'] = dist_df['distance (miles)']*dist_df['emission_factor']

# calculating total emssion from all flights
total_emission = dist_df['emission'].sum()
print(f'Total emission from all flights is {total_emission:.1f} ton CO2e') #change wording to kg if applicable

Total emission from all flights is 393.0 ton CO2e


## Final Result:
Total emission from all flights is **10876.8** kg of CO2.